# function def and uploading data

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import joblib
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings("ignore")

In [ ]:
def prepare_data(data, column_to_check, columns_to_drop=None, train_size=0.7):
    """
    Prepares the dataset by splitting it into training and testing sets.

    Parameters:
    - data: The dataset (DataFrame).
    - column_to_check: The target variable (dependent variable).
    - columns_to_drop: Optional; additional columns to drop before training.
    - train_size: The proportion of data to use for training (default 70%).

    Returns:
    - X_train: Training features
    - X_test: Testing features
    - y_train: Training target values
    - y_test: Testing target values
    """

    drop = []
    if columns_to_drop:
        # Ensure columns_to_drop is a list
        drop = [columns_to_drop] if isinstance(columns_to_drop, str) else columns_to_drop

    # X: Features (drop specified columns + target column)
    X = data.drop(drop + [column_to_check], axis=1)

    # y: Target variable (dependent variable)
    y = data[column_to_check]

    # Splitting data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=42, train_size=train_size
    )

    return X_train, X_test, y_train, y_test


# Function to calculate Root Mean Squared Error (RMSE)
def RMSE(y_, y_pred_):
    """
    Computes the Root Mean Squared Error (RMSE).

    Parameters:
    - y_: Actual values
    - y_pred_: Predicted values

    Returns:
    - RMSE value
    """
    return ((y_ - y_pred_) ** 2).mean() ** 0.5

In [ ]:
#upload train and valid data sets
df_train = pd.read_csv(file_path,
                 low_memory=False,
                 parse_dates=["saledate"])

df_valid = pd.read_csv(file_path,
                 low_memory=False,
                 parse_dates=["saledate"])

farm_data = pd.read_csv(file_path,
                 low_memory=False)

df_train.info()
df_valid.info()

# data preparation

In [ ]:
# ========================
# ========================
# Train preparation
# ========================
# ========================



# ========================
# Sort saledate and copy data
# ========================

# Sort by saledate in ascending order
df_train.sort_values(by=["saledate"], inplace=True, ascending=True)

# Copy dataset for preprocessing into df_tmp3
df_tmp3 = df_train.copy()

# ========================
# Extract Date Features
# ========================

df_tmp3["saleYear"]    = df_tmp3.saledate.dt.year
df_tmp3["saleMonth"]   = df_tmp3.saledate.dt.month
df_tmp3["saleDay"]     = df_tmp3.saledate.dt.day
df_tmp3["saleDayofweek"] = df_tmp3.saledate.dt.dayofweek
df_tmp3["saleDayofyear"] = df_tmp3.saledate.dt.dayofyear

# Drop original saledate column
df_tmp3.drop("saledate", axis=1, inplace=True)

# ========================
# Transform object columns to categories
# ========================

# Transform object columns to category
for col in df_tmp3.select_dtypes(['object']):
    df_tmp3[col] = df_tmp3[col].astype('category')

# ========================
# filling nulls
# ========================

# Initialize an empty list to store column names that contain "None or Unspecified"
columns_with_none_or_unspecified = []

# Iterate over categorical columns
for col in df_tmp3.select_dtypes(include=["category", "object"]):
    if df_tmp3[col].astype(str).str.contains("None or Unspecified", na=False).any():
        columns_with_none_or_unspecified.append(col)

# Fill nulls in columns that contain "None or Unspecified"
for col in columns_with_none_or_unspecified:
    df_tmp3[col] = df_tmp3[col].fillna("None or Unspecified")

# Find categorical columns with null values
categorical_columns_with_nulls = df_tmp3.select_dtypes(include=["category", "object"]).isnull().sum()
categorical_columns_with_nulls = categorical_columns_with_nulls[categorical_columns_with_nulls > 0]

# Convert to a list
categorical_columns_with_nulls_list = categorical_columns_with_nulls.index.tolist()

# Fill nulls in these categorical columns
for col in categorical_columns_with_nulls_list:
    if isinstance(df_tmp3[col].dtype, pd.CategoricalDtype):
        df_tmp3[col] = df_tmp3[col].cat.add_categories(["Unknown"])
    df_tmp3[col] = df_tmp3[col].fillna("Unknown")

# Fill nulls for MachineHoursCurrentMeter using its median
df_tmp3['MachineHoursCurrentMeter'] = df_tmp3['MachineHoursCurrentMeter'].fillna(
    df_tmp3['MachineHoursCurrentMeter'].median()
)

# ========================
# adding farm data
# ========================

# Ensure state names match by converting to lowercase
df_tmp3['state'] = df_tmp3['state'].str.lower()
farm_data['State'] = farm_data['State'].str.lower()

# Merge farm_data into df_tmp3 based on state and year
df_tmp3 = df_tmp3.merge(
    farm_data[['State', 'Year', 'Farms_per_Capita', 'Farms_per_Square_Mile']],
    left_on=['state', 'saleYear'],
    right_on=['State', 'Year'],
    how='left'
)

# Drop redundant merge columns (State, Year from farm_data)
df_tmp3.drop(columns=['State', 'Year'], inplace=True)

# ========================
# fixing columns types
# ========================

# 🔹 FIX: Convert 'state' back to categorical
df_tmp3['state'] = df_tmp3['state'].astype('category')

# ========================
# size_category column
# ========================

# Define a mapping for three levels + Unknown
size_category_mapping = {
    'Compact': 0, 'Mini': 0,  # Small
    'Small': 1, 'Medium': 1,  # Medium
    'Large': 2, 'Large / Medium': 2,  # Large
    'Unknown': -1  # Keep unknown separate
}

# Apply the mapping
df_tmp3['SizeCategory'] = df_tmp3['ProductSize'].map(size_category_mapping)

# ========================
# Prepare rare caegories lists
# ========================

# Define how many rare categories you want to extract per column
rare_counts = {
    'fiProductClassDesc': 34,
    'state': 27,
    'Undercarriage_Pad_Width': 11,
    'Stick_Length': 20
}

# Dictionary to store the rare categories for each column
rare_categories = {}

# Iterate over each column and number of rare categories
for col, n in rare_counts.items():
    # Count the occurrences of each unique value in the column,
    # sort them in ascending order, and take the lowest n values.
    rare_list = df_tmp3[col].value_counts().sort_values().head(n).index.tolist()
    rare_categories[col] = rare_list

# ========================
# Grouping rare categories
# ========================

for col, rare_list in rare_categories.items():
    # Check if the column is categorical
    if pd.api.types.is_categorical_dtype(df_tmp3[col]):
        # Add the "Other" category if it's not already present
        if "Other" not in df_tmp3[col].cat.categories:
            df_tmp3[col] = df_tmp3[col].cat.add_categories(["Other"])
    # Replace the values that are in the rare_list with "Other"
    df_tmp3.loc[df_tmp3[col].isin(rare_list), col] = "Other"

# ============================
# Apply Target Encoding (new)
# ============================
# Compute target mean encoding for fiSecondaryDesc in training data
if 'fiSecondaryDesc' in df_tmp3.columns:
    fiSecondaryDesc_target_map = df_tmp3.groupby('fiSecondaryDesc')['SalePrice'].mean()

    # Replace fiSecondaryDesc with the mean SalePrice of that category
    df_tmp3['fiSecondaryDesc'] = df_tmp3['fiSecondaryDesc'].map(fiSecondaryDesc_target_map)

# Compute target mean encoding for fiBaseModel in training data
if 'fiBaseModel' in df_tmp3.columns:
    fiBaseModel_target_map = df_tmp3.groupby('fiBaseModel')['SalePrice'].mean()

    # Replace fiBaseModel with the mean SalePrice of that category
    df_tmp3['fiBaseModel'] = df_tmp3['fiBaseModel'].map(fiBaseModel_target_map)

# Compute target mean encoding for fiModelDesc in training data
if 'fiModelDesc' in df_tmp3.columns:
    fiModelDesc_target_map = df_tmp3.groupby('fiModelDesc')['SalePrice'].mean()

    # Replace fiBaseModel with the mean SalePrice of that category
    df_tmp3['fiModelDesc'] = df_tmp3['fiModelDesc'].map(fiModelDesc_target_map)

# Compute target mean encoding for fiProductClassDesc in training data
if 'fiProductClassDesc' in df_tmp3.columns:
    fiProductClassDesc_target_map = df_tmp3.groupby('fiProductClassDesc')['SalePrice'].mean()

    # Replace fiProductClassDesc with the mean SalePrice of that category
    df_tmp3['fiProductClassDesc'] = df_tmp3['fiProductClassDesc'].map(fiProductClassDesc_target_map)

# ============================
# Coding categorical columns
# ============================

# Convert categorical columns to numeric codes
for col in df_tmp3.select_dtypes(['category']):
    df_tmp3[col] = df_tmp3[col].cat.codes

# ========================
# Adjust Prices Using CPI
# ========================

cpi_data = {
    1989: 124.0, 1990: 130.7, 1991: 136.2, 1992: 140.3, 1993: 144.5,
    1994: 148.2, 1995: 152.4, 1996: 156.9, 1997: 160.5, 1998: 163.0,
    1999: 166.6, 2000: 172.2, 2001: 177.1, 2002: 179.9, 2003: 184.0,
    2004: 188.9, 2005: 195.3, 2006: 201.6, 2007: 207.3, 2008: 215.3,
    2009: 214.5, 2010: 218.1, 2011: 224.9, 2012: 229.6
}

# Merge CPI values based on the sale year
df_tmp3["CPI"] = df_tmp3["saleYear"].map(cpi_data)

# Adjust SalePrice to 2012 equivalent
df_tmp3["SalePrice"] = df_tmp3["SalePrice"] * (cpi_data[2012] / df_tmp3["CPI"])

# Drop the CPI column if no longer needed
df_tmp3.drop("CPI", axis=1, inplace=True)

# ========================
# Create ValidYearMade Feature
# ========================

df_tmp3["ValidYearMade"] = ((df_tmp3["YearMade"] != 1000) & (df_tmp3["saleYear"] >= df_tmp3["YearMade"])).astype(int)



# ========================
# ========================
# Valid preparation
# ========================
# ========================



# ========================
# Sort saledate and copy data
# ========================

# Sort dataset by date
df_valid.sort_values(by=["saledate"], inplace=True, ascending=True)

# Copy dataset for preprocessing
df_tmp_valid3 = df_valid.copy()

# ========================
# Extract Date Features
# ========================
df_tmp_valid3["saleYear"] = df_tmp_valid3.saledate.dt.year
df_tmp_valid3["saleMonth"] = df_tmp_valid3.saledate.dt.month
df_tmp_valid3["saleDay"] = df_tmp_valid3.saledate.dt.day
df_tmp_valid3["saleDayofweek"] = df_tmp_valid3.saledate.dt.dayofweek
df_tmp_valid3["saleDayofyear"] = df_tmp_valid3.saledate.dt.dayofyear

# Drop original saledate column
df_tmp_valid3.drop("saledate", axis=1, inplace=True)

# ========================
# Transform object columns to categories
# ========================

# Transform object columns to category
for col in df_tmp_valid3.select_dtypes(['object']):
    df_tmp_valid3[col] = df_tmp_valid3[col].astype('category')

# ========================
# filling nulls
# ========================

# Filling nulls in columns_with_none_or_unspecified
for col in columns_with_none_or_unspecified:
    if isinstance(df_tmp_valid3[col].dtype, pd.CategoricalDtype):
        # Only add category if it's not already present
        if "None or Unspecified" not in df_tmp_valid3[col].cat.categories:
            df_tmp_valid3[col] = df_tmp_valid3[col].cat.add_categories(["None or Unspecified"])
    df_tmp_valid3[col] = df_tmp_valid3[col].fillna("None or Unspecified")

# Filling nulls in categorical_columns_with_nulls
for col in categorical_columns_with_nulls_list:
    if isinstance(df_tmp_valid3[col].dtype, pd.CategoricalDtype):
        df_tmp_valid3[col] = df_tmp_valid3[col].cat.add_categories(["Unknown"])
    df_tmp_valid3[col] = df_tmp_valid3[col].fillna("Unknown")

# Filling MachineHoursCurrentMeter nulls
df_tmp_valid3['MachineHoursCurrentMeter'] = df_tmp_valid3['MachineHoursCurrentMeter'].fillna(
    df_tmp_valid3['MachineHoursCurrentMeter'].median()
)

# ========================
# adding farm data
# ========================

# Ensure state names match by converting to lowercase
df_tmp_valid3['state'] = df_tmp_valid3['state'].str.lower()
farm_data['State'] = farm_data['State'].str.lower()

# Merge farm_data into df_tmp3 based on state and year
df_tmp_valid3 = df_tmp_valid3.merge(
    farm_data[['State', 'Year', 'Farms_per_Capita', 'Farms_per_Square_Mile']],
    left_on=['state', 'saleYear'],
    right_on=['State', 'Year'],
    how='left'
)

# Drop redundant merge columns (State, Year from farm_data)
df_tmp_valid3.drop(columns=['State', 'Year'], inplace=True)

# ========================
# fixing columns types
# ========================

# 🔹 FIX: Convert 'state' back to categorical
df_tmp_valid3['state'] = df_tmp_valid3['state'].astype('category')

# ========================
# SizeCategory column
# ========================

# Create SizeCategory column
df_tmp_valid3['SizeCategory'] = df_tmp_valid3['ProductSize'].map(size_category_mapping)

# ========================
# Grouping rare categories
# ========================

# Replace rare categories with "Other" (using your rare_categories dictionary)
for col, rare_list in rare_categories.items():
    # Check if the column is categorical
    if pd.api.types.is_categorical_dtype(df_tmp_valid3[col]):
        # Add the "Other" category if it's not already present
        if "Other" not in df_tmp_valid3[col].cat.categories:
            df_tmp_valid3[col] = df_tmp_valid3[col].cat.add_categories(["Other"])
    # Replace the values that are in the rare_list with "Other"
    df_tmp_valid3.loc[df_tmp_valid3[col].isin(rare_list), col] = "Other"

# ============================
# Apply Target Encoding (new)
# ============================

# Apply the trained target encoding to validation data
if 'fiSecondaryDesc' in df_tmp_valid3.columns:
    # Fill missing categories with the overall mean SalePrice from training data
    overall_mean_price = df_tmp3['SalePrice'].mean()

    df_tmp_valid3['fiSecondaryDesc'] = df_tmp_valid3['fiSecondaryDesc'].map(fiSecondaryDesc_target_map)

    # Handle unseen categories in validation set by filling with overall mean SalePrice
    df_tmp_valid3['fiSecondaryDesc'].fillna(overall_mean_price, inplace=True)

# Apply the trained target encoding to validation data
if 'fiBaseModel' in df_tmp_valid3.columns:
    # Fill missing categories with the overall mean SalePrice from training data
    overall_mean_price = df_tmp3['SalePrice'].mean()

    df_tmp_valid3['fiBaseModel'] = df_tmp_valid3['fiBaseModel'].map(fiBaseModel_target_map)

    # Handle unseen categories in validation set by filling with overall mean SalePrice
    df_tmp_valid3['fiBaseModel'].fillna(overall_mean_price, inplace=True)

# Apply the trained target encoding to validation data
if 'fiModelDesc' in df_tmp_valid3.columns:
    # Fill missing categories with the overall mean SalePrice from training data
    overall_mean_price = df_tmp3['SalePrice'].mean()

    df_tmp_valid3['fiModelDesc'] = df_tmp_valid3['fiModelDesc'].map(fiModelDesc_target_map)

    # Handle unseen categories in validation set by filling with overall mean SalePrice
    df_tmp_valid3['fiModelDesc'].fillna(overall_mean_price, inplace=True)

# Apply the trained target encoding to validation data
if 'fiProductClassDesc' in df_tmp_valid3.columns:
    # Fill missing categories with the overall mean SalePrice from training data
    overall_mean_price = df_tmp3['SalePrice'].mean()

    df_tmp_valid3['fiProductClassDesc'] = df_tmp_valid3['fiProductClassDesc'].map(fiProductClassDesc_target_map)

    # Handle unseen categories in validation set by filling with overall mean SalePrice
    df_tmp_valid3['fiProductClassDesc'].fillna(overall_mean_price, inplace=True)

# ============================
# Coding categorical columns
# ============================

# Convert all categorical columns in df_tmp_valid3 to numeric codes
for col in df_tmp_valid3.select_dtypes(['category']):
    df_tmp_valid3[col] = df_tmp_valid3[col].cat.codes

# ========================
# Create ValidYearMade Feature
# ========================

# Create ValidYearMade column
df_tmp_valid3["ValidYearMade"] = ((df_tmp_valid3["YearMade"] != 1000) &
                                  (df_tmp_valid3["saleYear"] >= df_tmp_valid3["YearMade"])).astype(int)

# model

In [ ]:
#model train
X_train, X_test, y_train, y_test = prepare_data(df_tmp3, 'SalePrice', train_size=0.8)

model = RandomForestRegressor(
    n_estimators=200,
    max_features=0.5,
    random_state = 42,
    min_samples_leaf=1,
    n_jobs=1,
    min_samples_split=14,
    max_samples=None,
    max_depth=25
)
model = model.fit(X_train, y_train)

y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

train_rmse = RMSE(y_train, y_train_pred)
test_rmse = RMSE(y_test, y_test_pred)

print(f"train: {train_rmse}")
print(f"test: {test_rmse}")

df_tmp_valid3['SalePrice'] = model.predict(df_tmp_valid3)

new_data = df_tmp_valid3[['SalesID', 'SalePrice']]

new_data.to_csv(file_path, index=False)